## WTI Oil Prices
* Lag features
#### Economic growth
* US dollar index 
* S&P 500 index 
* US Federal Reserve rate
* Volatility index (VIX)
* Consumer price index (CPI).

source: https://savearchive.zbw.eu/bitstream/11159/5075/1/1747444766.pdf

#### Weather data
source: https://www.eia.gov/finance/markets/crudeoil/spot_prices.php
#### Supply and Demand
* OPEC

source: https://www.investopedia.com/articles/investing/072515/top-factors-reports-affect-price-oil.asp


## Energy Information Administration (EIA) Api
| Features | Frequence | Data Until |
|---|---|---|
| WTI | Daily | up-to-date |
| Oil Production | Monthly | July 2025|
|Weekly Input Utilization | Weekly | up-to-date |
| Gasoline Price | Weekly | up-to-date |
|Imports & Exports | Weekly | up-to-date |
|Crude Oil Supplied | Monthly | July 2025 |

In [24]:
from src.oil_optimization.utils.io_helpers import read_yaml

config = read_yaml('config/config.yml')

data_dir = config['data_ingestion']['root_dir']

In [28]:
import pandas as pd
import requests
from dotenv import dotenv_values

secrets = dotenv_values('.env')

api_calls  = {
    'wti_prices':{
        'url':'https://api.eia.gov/v2/petroleum/pri/spt/data',
        'payload':{
                    'api_key':secrets['EIA_API_KEY'],
                    'facets[product][]':'EPCWTI',
                    'start':'2015-01-01',
                    'sort[0][column]':'period',
                    'sort[0][direction]':'asc',
                    'frequency':'daily',
                    'data[0]':'value'
                }
    },
    'oil_production':{
        'url':'https://api.eia.gov/v2/petroleum/crd/crpdn/data',
        'payload':{
                    'api_key':secrets['EIA_API_KEY'],
                    'facets[product][]':'EPC0',
                    'start':'2015-01-01',
                    'sort[0][column]':'period',
                    'sort[0][direction]':'asc',
                    'data[0]':'value'
                }
    },
    'input_utilization':{
        'url':'https://api.eia.gov/v2/petroleum/pnp/wiup/data',
        'payload':{
                    'api_key':secrets['EIA_API_KEY'],
                    'facets[product][]':'EPC0',
                    'start':'2015-01-01',
                    'sort[0][column]':'period',
                    'sort[0][direction]':'asc',
                    'data[0]':'value'
                }
    },
    'gasoline_price':{
        'url':'https://api.eia.gov/v2/petroleum/pri/gnd/data',
        'payload':{
                    'api_key':secrets['EIA_API_KEY'],
                    'facets[product][]':'EPM0',
                    'start':'2015-01-01',
                    'sort[0][column]':'period',
                    'sort[0][direction]':'asc',
                    'data[0]':'value',
                }
    },
    'imports_and_exports':{
        'url':'https://api.eia.gov/v2/move/wkly/data',
        'payload':{
                    'api_key':secrets['EIA_API_KEY'],
                    'facets[product][]':'EPC0',
                    'start':'2015-01-01',
                    'sort[0][column]':'period',
                    'sort[0][direction]':'asc',
                    'data[0]':'value'
                }
    },
    'oil_supplied':{
        'url':'https://api.eia.gov/v2/petroleum/cons/psup/data',
        'payload':{
                    'api_key':secrets['EIA_API_KEY'],
                    'facets[product][]':'EPC0',
                    'start':'2015-01-01',
                    'sort[0][column]':'period',
                    'sort[0][direction]':'asc',
                    'data[0]':'value'
                }
    }}

In [34]:
import time

for file_name,params_dict in api_calls.items():
    r = requests.get(params_dict['url'], params=params_dict['payload'])
    df = pd.DataFrame(r.json()['response']['data'])
    df.to_csv(f'{data_dir}/raw/{file_name}.csv')
    time.sleep(3)


KeyError: 'response'

## Stock Market Indexes
* S&P 500 Energy

In [56]:
import yfinance as yf

In [ ]:
energy = yf.download('^GSPE',start='2015-01-01')    

C:\Users\User\AppData\Local\Temp\ipykernel_1752\627084361.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  energy = yf.download('^GSPE',start='2015-01-01')
[*********************100%***********************]  1 of 1 completed


In [78]:
energy.Close.reset_index().to_csv('data/energy.csv',index=False)

In [13]:
import pprint
from src.oil_optimization.utils.io_helpers import read_yaml
content = read_yaml('config/config.yml')
pprint.pprint(content)

{'data_ingestion': {'api_endpoint': ['https://api.eia.gov/v2/crude-oil-imports/data',
                                     'https://api.eia.gov/v2/petroleum/pri/spt/data'],
                    'macro_indicators_urls': ['https://contenido.bce.fin.ec/documentos/informacioneconomica/indicadores/real/PrecioPetroleoCrudoEcuatoriano.html',
                                              'https://contenido.bce.fin.ec/documentos/informacioneconomica/indicadores/real/Inflacion.html'],
                    'oil_reports_url': ['https://contenido.bce.fin.ec/documentos/Estadisticas/Hidrocarburos/SerieCifrasPetroleras.xlsx'],
                    'root_dir': 'data',
                    'yfinance': {'start_date': '2015-01-01',
                                 'ticker': '^GSPE'}}}


## Federeal Reserve Bank of St. Louis (FRED)